In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [5]:
def get_existing_certs(distinct_urls,folder):
    successes = []
    data_path = folder+'/'
    for url in distinct_urls:
        try:
            f = open(data_path+url)    
            if f.readline() != '':
                successes.append(url)
        except:
            pass
    return successes

In [3]:
path = "url_datasets/majestic_million.csv"
df = pd.read_csv(path)
urls = df['Domain']

In [4]:
how_many = 50000
urls = urls[:50000]

In [7]:
data_path = 'benign_data'
successes = get_existing_certs(urls,data_path)
len(successes)

38010

In [8]:
failures = list(set(urls) - set(successes))
len(failures)

11990

In [25]:
def construct_database(urls,data_path):
    
    #log which crt files are not empty
    successes = []
    for url in urls:
        try:
            f = open(data_path+url)    
            if f.readline() != '':
                successes.append(url)
        except:
            pass
    
    subjects = []
    verifications = []
    issuers = []
    incomplete = []
    for url in successes:
        f = open(data_path+url)
        encoding_lines = []
        line = f.readline()
        began=False
        subject=False
        issuer=False
        verified=False
        while line != '':

            if issuer:
                issuers.append(line)
                issuer=False
            if "0 s" in line and subject == False:
                subject=True
                subjects.append(line)
                issuer = True
            if not began and "-----BEGIN CERTIFICATE-----" in line:
                began = True
            if began:
                if "-----END CERTIFICATE-----" in line:
                    began = False
                encoding_lines.append(line)
            if "Verify return code:" in line:
                if verified:
                    #print(url)
                    pass
                else:
                    verifications.append(line)
                    verified=True

            line = f.readline()
            
        if not verified:
            #print("FAILED: "+url)
            verifications.append('Verify return code: None (none)')
        if subject == False:
            incomplete.append(url)
            issuers.append('None')
            subjects.append('None')
        f.close()   
        
    new_issuers = [i.split(' = ') for i in issuers]
    new_issuers_O = []
    for i in new_issuers:
        done = False
        if i[0] == 'None':
            new_issuers_O.append('None')
            done = True
        for z,j in enumerate(i):
            if done:
                break
            if ", O" == j[-3:]:
                new_issuers_O.append(i[z+1][:-4])
                done = True
            elif j.endswith("i:O"):
                new_issuers_O.append(i[z+1][:-4])
                done = True
        if not done:
            new_issuers_O.append('MISSING')
            
    new_issuers_CN = []
    for i in new_issuers:
        done = False
        if i[0] == 'None':
            new_issuers_CN.append('None')
            done = True    
        for z,j in enumerate(i):
            if done:
                break
            if ", CN" == j[-4:]:
                new_issuers_CN.append(i[z+1][:-1])
                done = True
            elif j.endswith("i:CN"):
                new_issuers_CN.append(i[z+1][:-1])
                done = True
        if not done:
            new_issuers_CN.append('MISSING')       
            
    new_verifications = [v[5:].split(' ')[3] for v in verifications]
    
    new_subjects = [s[5:].split(', ') for s in subjects]
    CN, C, ST, L = [],[],[],[]
    for url in new_subjects:
        found_cn, found_c, found_st, found_l = False,False,False,False
        for attr in url:
            if attr.startswith('CN ='):       
                if found_cn:
                    print(url)
                else:
                    CN.append(attr[5:-1])
                found_cn=True
            elif attr.startswith('C ='):
                C.append(attr[4:])
                found_c=True
            elif attr.startswith('ST ='):
                ST.append(attr[5:])
                found_st=True
            elif attr.startswith('L ='):
                L.append(attr[4:])
                found_l=True
        if not found_cn:
            CN.append(None)
        if not found_c:
            C.append(None)
        if not found_st:
            ST.append(None)
        if not found_l:
            L.append(None)
            
    df = pd.DataFrame()
    df['url'] = successes
    df['CN'] = CN
    df['C'] = C
    df['ST'] = ST
    df['L'] = L
    
    df['Issuer'] = new_issuers_O
    df['Issuer_CN'] = new_issuers_CN
    df['VerCode'] = new_verifications
    return df

In [26]:
df = construct_database(urls,data_path+'/')

['C = GB', 'ST = Cambridge', 'O = Darwin Online', 'CN = *.darwin-online.org.uk', 'CN = darwin-online.org.uk', 'CN = wallace-online.org', 'emailAddress = webmaster@darwin-online.org.uk\n']
['C = GB', 'ST = Cambridge', 'O = Darwin Online', 'CN = *.darwin-online.org.uk', 'CN = darwin-online.org.uk', 'CN = wallace-online.org', 'emailAddress = webmaster@darwin-online.org.uk\n']


In [27]:
df

,url,CN,C,ST,L,Issuer,Issuer_CN,VerCode
0,google.com,*.google.com,US,California,Mountain View,Google Trust Services,GTS CA 1O1,0
1,youtube.com,*.google.com,US,California,Mountain View,Google Trust Services,GTS CA 1O1,0
2,twitter.com,twitter.com,US,California,San Francisco,DigiCert Inc,DigiCert TLS RSA SHA256 2020 CA1,0
3,linkedin.com,www.linkedin.com,US,California,Sunnyvale,DigiCert Inc,DigiCert SHA2 Secure Server CA,0
4,microsoft.com,microsoft.com,None,None,None,Microsoft Corporation,Microsoft RSA TLS CA 01,0
...,...,...,...,...,...,...,...,...
38005,servicemax.com,*.servicemax.com,US,California,Pleasanton,DigiCert Inc,DigiCert TLS RSA SHA256 2020 CA1,0
38006,malegramed.com,sni.cloudflaressl.com,US,CA,San Francisco,"""Cloudflare, Inc.""",Cloudflare Inc ECC CA-3,0
38007,sildenafilyes.com,sni.cloudflaressl.com,US,CA,San Francisco,"""Cloudflare, Inc.""",Cloudflare Inc ECC CA-3,0
38008,pillmy.com,sni.cloudflaressl.com,US,CA,San Francisco,"""Cloudflare, Inc.""",Cloudflare Inc ECC CA-3,0


In [21]:
len(CNs)

38012